In [1]:
!pip install requests pandas
!pip install statsmodels

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [3]:
df = pd.read_csv('FAOSTAT_nozer.csv')

#manually select all outside crops (exclude greenhouse crops):

selected_items = ['Blueberries', 'Broad beans and horse beans, dry', 'Cantaloupes and other melons',
 'Leeks and other alliaceous vegetables', 'Pumpkins, squash and gourds', 'Poppy seed',
 'Cherries', 'Grapes', 'Anise, badian, coriander, cumin, caraway, fennel and juniper berries, raw',
 'Maize (corn)', 'Raspberries', 'Other berries and fruits of the genus vaccinium n.e.c.',
 'Linseed', 'Currants', 'Rape or colza seed', 'Plums and sloes', 'Beans, dry', 'Peas, dry',
 'Broad beans and horse beans, green', 'Sour cherries', 'Rye', 'Asparagus', 'Flax, raw or retted',
 'Oats', 'Other vegetables, fresh n.e.c.', 'Peas, green', 'Carrots and turnips', 'Cabbages',
 'Barley', 'Onions and shallots, dry (excluding dehydrated)', 'Apples', 'Wheat', 'Sugar beet',
 'Potatoes']

crop_df = df[df['Item'].isin(selected_items)][['Item', 'Year', 'Value']]

                                                                              #exclude any crops with any 0 values:
items_with_zero = crop_df[crop_df['Value'] == 0.0]['Item'].unique()
crop_df = crop_df[~crop_df['Item'].isin(items_with_zero)][['Item', 'Year', 'Value']]

In [4]:
df_X = pd.read_csv('final_yearly_merged_data.csv', usecols=range(10))
print(df_X.columns)
# One-hot encode the crops in crop_df, ensuring 'Year' is of the same type for merging
crop_df['Year'] = crop_df['Year'].astype(float)
crop_df_one_hot = pd.get_dummies(crop_df, columns=['Item'])

# Merge crop data with weather statistics based on 'Year'
df_merged = crop_df_one_hot.merge(df_X, on='Year', how='inner')

# Separate features (X) and target (y)
X = df_merged.drop(columns=['Year', 'Value'])
y = df_merged['Value']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify numeric columns for scaling (excluding one-hot encoded columns)
numeric_columns = X_train.select_dtypes(include=['float64', 'int64']).columns

# Initialize and fit the scaler on the training set's numeric columns
scaler = StandardScaler()
X_train[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])

# Apply the same transformation to the test set's numeric columns
X_test[numeric_columns] = scaler.transform(X_test[numeric_columns])

# Convert X and y back to values
X_train, X_test, y_train, y_test = X_train.values, X_test.values, y_train.values, y_test.values

Index(['Year', 'Cloud cover', 'potential evapo-transpiration',
       'Precipitation rate', 'Minimum 2m temperature', 'Mean 2m temperature',
       'Maximum 2m temperature', 'Vapour pressure', 'Frost days', 'Wet days'],
      dtype='object')


In [5]:
def shift_target_by_year(df, crop_column, target_column):
    """
    Baseline model, predicts previous year yield of a crop for a crop
    """
    df_sorted = df.sort_values(by=[crop_column, 'Year'])
    df_sorted[target_column + '_previous'] = df_sorted.groupby(crop_column)[target_column].shift(1)
    return df_sorted


df = shift_target_by_year(crop_df, crop_column='Item', target_column='Value')

df = df.dropna(subset=['Value_previous'])

# Filter out the data for the year 1961
df_filtered = df[df['Year'] != 1961]

# The 'Value_previous' column will be used as the prediction
y_true = df_filtered['Value']
y_pred = df_filtered['Value_previous']
mae_baseline = mean_absolute_error(y_true, y_pred)
mse_baseline = mean_squared_error(y_true, y_pred)
rmse_baseline = np.sqrt(mse_baseline)
r2_baseline = r2_score(y_true, y_pred)

In [6]:
# Train a single LinearRegression model on all features
model = LinearRegression()
model.fit(X_train, y_train)

# Predict using the test set and ensure predictions are non-negative
y_pred = model.predict(X_test)
y_pred = np.maximum(y_pred, 0)

# Calculate the mean squared error
final_mse = mean_squared_error(y_test, y_pred)

# Display results
print(f"Baseline Model MSE (Previous Year's Yield Prediction): {mse_baseline / 1e9}")
print(f"Single Model MSE: {final_mse / 1e9}")

Baseline Model MSE (Previous Year's Yield Prediction): 54.59361596668838
Single Model MSE: 130.05311206175182


In [7]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Calculating MAE
final_mae = mean_absolute_error(y_test, y_pred)

# Calculating MSE (already computed)
final_mse = mean_squared_error(y_test, y_pred)

# Calculating RMSE
final_rmse = np.sqrt(final_mse)

# Calculating R^2
final_r2 = r2_score(y_test, y_pred)

# Display results
print(f"Mean Absolute Error (MAE): {final_mae/1000}")
print(f"Mean Squared Error (MSE): {final_mse/1000000000}")
print(f"Root Mean Squared Error (RMSE): {final_rmse/1000}")
print(f"R^2 Score: {final_r2}")

Mean Absolute Error (MAE): 144.20910151996554
Mean Squared Error (MSE): 130.05311206175182
Root Mean Squared Error (RMSE): 360.62877320279347
R^2 Score: 0.9626431886647503


In [8]:
print(f"MAE: {mae_baseline/1000}")
print(f"MSE: {mse_baseline/1000000000}")
print(f"RMSE: {rmse_baseline/1000}")
print(f"R²: {r2_baseline}")

MAE: 62.627591332540156
MSE: 54.59361596668838
RMSE: 233.6527679414228
R²: 0.9793561133772898


In [12]:
X_train = X
y_train = y

# Identify numeric columns for scaling (excluding one-hot encoded columns)
numeric_columns = X_train.select_dtypes(include=['float64', 'int64']).columns

# Initialize and fit the scaler on the training set's numeric columns
scaler = StandardScaler()
X_train[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])
X_train, y_train= X_train.values, y_train.values

In [13]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [14]:
#save model

ensemble_data = {
    'model': model,
    'scaler': scaler
}

# Save the ensemble data
joblib.dump(ensemble_data, 'ensemble_models.joblib_final')
print("Ensemble models saved as 'ensemble_models.joblib_final'")

Ensemble models saved as 'ensemble_models.joblib_final'
